In [61]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [62]:
# !pip install pydantic --upgrade


In [63]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

from langchain.tools import tool

In [64]:

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI


In [65]:
class Extract_toolArgs(BaseModel):
    a: str = Field(description="Company name")
    b: str = Field(description="item")
    c: str = Field(description="Year_1")
    d: str = Field(description="Year_2")

In [66]:
class Extracted_toolArgs(BaseModel):
    year : list = Field(description="List of years")
    value : list = Field(discription = "List of Values")

In [67]:
import json
@tool(args_schema=Extract_toolArgs)
def Extract_Tool(a: str, b: str, c:str, d:str) -> Extract_toolArgs:
    """extracts specific data for a company from Year_1 to Year_2."""
    with open("data_soo.json","r") as json_file:
        data = json.load(json_file)
    
    a = a.upper()
    b = b.lower()
    json_data = next(item for item in data[a] if item["Item"] == b)
    
    year_data =[]
    cash_data = []
    
    # Ensure c and d represent years in the correct order
    if int(c) > int(d):
        c, d = d, c
    
    for year in range(int(c), int(d) + 1):
        value = json_data.get(str(year))
        if value is not None:
            cash_data.append(value)
            year_data.append(year)

            
    
    if len(cash_data) == 0:
        raise ValueError(f"Data for the years {c} to {d} is not available.")
    
    return cash_data,year_data




In [68]:
class GrowthToolArgs(BaseModel):
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of sales data")


In [69]:
@tool(args_schema=GrowthToolArgs)
def Growth_Tool(sales_data: list, year_data: list) -> list[float]:
    """
    Calculates year-on-year (YoY) growth for 'Net Sales' for a specific company.
    This function receives year data and sales data list.

    Arguments:
    - sales_data: List of dictionaries containing year and corresponding sales.
    - year_data: List of years corresponding to the sales data.
    
    Returns:
    - A list of floats representing the YoY growth percentage for each year compared to the previous year.
    """
    # Ensure there is sufficient data for year-on-year calculation
    if len(sales_data) < 2:
        return ["Not enough data to calculate year-on-year sales growth."]
    
    yoy_growth = []
    
    # Sort the sales data by year to ensure proper calculation
    sales_data_sorted = sorted(zip(year_data, sales_data), key=lambda x: x[0])

    # Extract sorted years and sales
    sorted_years, sorted_sales = zip(*sales_data_sorted)

    for i in range(1, len(sorted_years)):
        current_year = sorted_years[i]
        previous_year = sorted_years[i - 1]

        current_year_sales = sorted_sales[i]
        previous_year_sales = sorted_sales[i - 1]

        if previous_year_sales is not None and current_year_sales is not None:
            # Calculate YoY growth
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(growth)
        else:
            yoy_growth.append(None)  # If data is missing
    
    return yoy_growth


In [70]:
class GrossMarginToolArgs(BaseModel):
    gross_margin_data : list = Field(description="list of Gross margin data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of Gross margin data")

In [71]:
@tool(args_schema=GrossMarginToolArgs)
def GrossmmarginpercTool(gross_margin_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates percentage of Gross Margin.
    This function receives year data and sales data list and Gross margin.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    -gross_margin_data: List of corresponding gross margin data.
    
    Returns:
    - A list of floats representing the YoY gross margin percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(gross_margin_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, gross_margin_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_gross_margins, sorted_sales = zip(*data_sorted)

    yoy_gross_margin_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            gross_margin_percentage = (sorted_gross_margins[i] / sorted_sales[i]) * 100
            yoy_gross_margin_percentage.append(gross_margin_percentage)
        else:
            yoy_gross_margin_percentage.append(None)  # If data is missing or invalid

    return yoy_gross_margin_percentage

In [72]:
tools = [Extract_Tool,Growth_Tool, GrossmmarginpercTool]

In [73]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key_string)

In [74]:
prompt = hub.pull("hwchase17/structured-chat-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [75]:
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

In [76]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [77]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

In [78]:
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Extract_Tool and Growth_Tool"
memory.chat_memory.add_message(SystemMessage(content=initial_message))


In [79]:
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Gross Margin",
    "c": "2021",
    "d": "2024"
  }
}([152836, 170782, 169148, 180683], [2021, 2022, 2023, 2024])Action:
```
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Net Sales",
    "c": "2021",
    "d": "2024"
  }
}
```([365817, 394328, 383285, 391035], [2021, 2022, 2023, 2024])Action:
```
{
  "action": "GrossmmarginpercTool",
  "action_input": {
    "gross_margin_data": [152836, 170782, 169148, 180683],
    "sales_data": [365817, 394328, 383285, 391035],
    "year_data": [2021, 2022, 2023, 2024]
  }
}
```[41.77935962516778, 43.30963056136009, 44.13112957720756, 46.206349815233935]Action:
```
{
  "action": "Final Answer",
  "action_input": "The gross margin percentages for Apple from 2021 to 2024 are as follows: \n- 2021: 41.78%\n- 2022: 43.31%\n- 2023: 44.13%\n- 2024: 46.21%\n\nThe trend shows a steady increase in the gross margin 

In [ ]:
# !pip install pydantic==1.10.11



  Obtaining dependency information for pydantic==1.10.11 from https://files.pythonhosted.org/packages/93/63/c2b91f7482bf8e5b7166dba6558287d906587f269158b8fad1146031fdc3/pydantic-1.10.11-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.8 MB/s eta 0:00:00a 0:00:01m
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydantic-2.10.3
ERROR: pip's dependency resolver does not currently take into account

In [ ]:
# prompt = hub.pull("hwchase17/openai-tools-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# agent = create_tool_calling_agent(
#     llm=llm,  # Language model to use
#     tools=tools,  # List of tools available to the agent
#     prompt=prompt,  # Prompt template to guide the agent's responses
# )

In [ ]:
# agent_executor = AgentExecutor.from_agent_and_tools(
#     agent=agent,  # The agent to execute
#     tools=tools,  # List of tools available to the agent
#     verbose=True,  # Enable verbose logging
#     handle_parsing_errors=True,  # Handle parsing errors gracefully
# )

In [ ]:
# response = agent_executor.invoke({"input": " extract net sales of Apple 2019 to 2023 "})



> Entering new AgentExecutor chain...

Invoking: `Extract_Tool` with `{'a': 'Apple', 'b': 'Net Sales', 'c': '2019', 'd': '2023'}`


([260174, 274515, 365817, 394328, 383285], [2019, 2020, 2021, 2022, 2023])The net sales of Apple from 2019 to 2023 are as follows:

- 2019: $260,174 million
- 2020: $274,515 million
- 2021: $365,817 million
- 2022: $394,328 million
- 2023: $383,285 million

> Finished chain.


In [ ]:
# response = agent_executor.invoke({"input": "generate some insights of sales growth of apple"})



> Entering new AgentExecutor chain...

Invoking: `Extract_Tool` with `{'a': 'Apple', 'b': 'Net Sales', 'c': '2015', 'd': '2022'}`


([233715, 215639, 229234, 265595, 260174, 274515, 365817, 394328], [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])
Invoking: `Growth_Tool` with `{'sales_data': [233715, 215639, 229234, 265595, 260174, 274515, 365817, 394328], 'year_data': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]}`


[-7.734206191301371, 6.304518199398068, 15.861957650261305, -2.041077580526742, 5.512080376978483, 33.25938473307469, 7.7937876041846055]Here are some insights on the sales growth of Apple from 2015 to 2022:

1. **2015 to 2016**: There was a decline in sales, with a year-on-year (YoY) growth rate of -7.73%. This indicates a decrease in net sales for Apple in 2016 compared to 2015.

2. **2016 to 2017**: Apple saw a rebound with a YoY growth rate of 6.30%. This suggests a recovery in sales after the decline in the previous year.

3. **2017 to 2018**: The growth contin

In [ ]:
# print(response)

{'input': 'give me insights on sales growth from 2021 to 2024 for apple', 'output': "Here are the insights on Apple's sales growth from 2021 to 2024:\n\n- **2021 to 2022**: Apple experienced a growth in net sales of approximately **7.79%**.\n- **2022 to 2023**: There was a decline in net sales by about **2.80%**.\n- **2023 to 2024**: Sales increased slightly with a growth of around **2.02%**.\n\nOverall, Apple's net sales showed fluctuations over these years, with a notable growth in 2022 followed by a slight decline in 2023 and a small recovery in 2024."}
